In [268]:
import librosa
import numpy as np
import shutil
import os
import re
from joblib import Parallel, delayed
import multiprocessing 
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from scipy import stats
import sklearn.metrics as metrices
import seaborn as sns
import librosa.display
from sklearn.neighbors import KNeighborsClassifier as KNN

In [305]:
raw_dir='recordings/selected_recordings/'
silence_removed_dir='recordings/silence_removed/'
unified_dir='recordings/unified_equal/'

In [306]:
mfcc_window_size=0.03
mfcc_stride_size=0.007
mfcc_num_of_features=13

In [307]:
def extract_mfcc(fname):
    y,sr=librosa.load(unified_dir+fname)
    result=librosa.feature.mfcc(y=y,n_mfcc=mfcc_num_of_features,hop_length=int(mfcc_window_size*sr), n_fft=int(mfcc_stride_size*sr))
    #result=librosa.util.normalize(result)
    #Add label as well
    if 'mandarin' in fname:
        label=0
    elif 'english' in fname:
        label=1
    elif 'spanish' in fname:
        label=2
    else:
        label=-1
    return [result,label]

In [308]:
result=Parallel(n_jobs=12)(delayed(extract_mfcc)(fname) for fname in os.listdir(unified_dir))

In [309]:
X=[]
y=[]
for i in result:
    X.append(i[0].T)
    y.append(i[1])

In [310]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.1)

In [311]:
y_combined=[]
Xs=[]
for i in range(len(X_train)):
    print(X_train[i].shape)
    tmp=X_train[i]
    ytmp=[y_train[i]]*X_train[i].shape[0]
    y_combined+=ytmp
    Xs.append(tmp)

(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)
(667, 13)


In [312]:
Xtrain=np.concatenate(Xs,axis=0)

In [313]:
knnm=KNN(n_neighbors=3)
knnm.fit(Xtrain,np.array(y_combined))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [314]:
olables=[]
rawre=[]
for i in range(len(X_test)):
    sre=knnm.predict(X_test[i])
    rawre.append(sre)
    olables.append(stats.mode(sre)[0][0])

In [315]:
np.sum(rawre[0]==0)

263

In [316]:
rawre[0].shape

(667,)

In [317]:
olables

[0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0]

In [318]:
y_test

[0, 2, 0, 2, 0, 0, 1, 0, 1, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 1]

In [319]:
metrices.accuracy_score(olables,y_test)

0.55